In [14]:
from datasets import load_dataset
import os

os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
print('正在下载数据集')
dataset = load_dataset('lansinuote/ChnSentiCorp')

print('数据集下载完成')
print(dataset)

print(dataset['train'][0])

正在下载数据集


'(ReadTimeoutError("HTTPSConnectionPool(host='hf-mirror.com', port=443): Read timed out. (read timeout=10)"), '(Request ID: 9ed70e6e-aae8-484d-98d3-92932da3d5e8)')' thrown while requesting HEAD https://hf-mirror.com/datasets/lansinuote/ChnSentiCorp/resolve/main/README.md
Retrying in 1s [Retry 1/5].


数据集下载完成
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 9600
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1200
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1200
    })
})
{'text': '选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般', 'label': 1}


In [15]:
from transformers import AutoTokenizer

In [16]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-chinese')

In [17]:
tokenizer

BertTokenizerFast(name_or_path='bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [18]:
def process_function(examples):
    return tokenizer(examples['text'],max_length=128,truncation=True,padding='max_length')

In [19]:
tokenzied_datasets = dataset.map(process_function,batched=True)

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

In [20]:
tokenzied_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9600
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1200
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1200
    })
})

In [21]:
from transformers import DataCollatorWithPadding

In [24]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [26]:
samples = [tokenzied_datasets['train'][i] for i in range(8)]

In [27]:
samples = [{k:v for k,v in s.items() if k != 'text'} for s in samples]

In [28]:
batch = data_collator(samples)

In [29]:
batch = {k:v.to("cuda") for k,v in batch.items()}

In [31]:
batch['input_ids'].shape

torch.Size([8, 128])

In [33]:
batch['input_ids'].device

device(type='cuda', index=0)